<a href="https://colab.research.google.com/github/badrilosaberidze/Facial-expression-recognition-challenge/blob/main/Model_Experiment_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login into wandb And essential imports

In [1]:
!pip install -q wandb
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from PIL import Image


wandb.init(project="facial-expression-recognition", config={
    "epochs": 15,
    "batch_size": 64,
    "lr": 1e-3,
    "architecture": "SimpleCNN"
}, name = 'deeper-cnn-dropout0.3-LRDecay')
config = wandb.config

wandb: Currently logged in as: blosa22 (blosa22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Mount to drive and downlaod dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset
!kaggle competitions download -c challengeas-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip


403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/competitions/data/download-all/challengeas-in-representation-learning-facial-expression-recognition-challenge
replace example_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace example_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


Read Data And Preproccessing

In [2]:
class FERDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.images = dataframe['pixels'].apply(lambda x: np.array(x.split(), dtype='float32').reshape(48, 48))
        self.labels = dataframe['emotion'].values
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images.iloc[idx]).convert("L")
        if self.transform:
            img = self.transform(img)
        return img, self.labels[idx]

## this was for augmentation , which in fact didnt worked really well
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(48, scale=(0.9, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

#dataset = FERDataset("train.csv", transform=transform)
#train_size = int(0.8 * len(dataset))
#val_size = len(dataset) - train_size
#train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])
#train_loader = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)
#val_loader = DataLoader(val_ds, batch_size=config.batch_size)


In [3]:
df = pd.read_csv("train.csv")


from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['emotion'])

train_ds = FERDataset(train_df, transform=val_transform)
val_ds = FERDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=config.batch_size)


Base CNN

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        import torch.nn as nn

        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),

            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),

            nn.Flatten(),
            nn.Linear(256 * 3 * 3, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 7)
        )


    def forward(self, x):
        return self.net(x)

model = SimpleCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


SimpleCNN(
  (net): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.3, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.3, inplace=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout(p=0.3, inplace=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Dropout(p=0.3, inplace=False)
    (16): Flatten(start_dim=1, end_dim=-1)
    (17): Linear(in_features=2304, out_features=128, bias=Tru

Train And Log Into wandb

In [5]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

wandb.watch(model, log="all", log_freq=100)


for epoch in range(config.epochs):
    model.train()
    running_loss, correct = 0.0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    train_acc = correct / len(train_loader.dataset)
    train_loss = running_loss / len(train_loader)

    scheduler.step()

    wandb.log({"train_loss": train_loss, "train_acc": train_acc, "epoch": epoch + 1})
    print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")


Epoch 1, Loss: 1.7861, Acc: 0.2570
Epoch 2, Loss: 1.6412, Acc: 0.3542
Epoch 3, Loss: 1.5119, Acc: 0.4144
Epoch 4, Loss: 1.4287, Acc: 0.4494
Epoch 5, Loss: 1.3691, Acc: 0.4698
Epoch 6, Loss: 1.3150, Acc: 0.4991
Epoch 7, Loss: 1.2872, Acc: 0.5090
Epoch 8, Loss: 1.2580, Acc: 0.5194
Epoch 9, Loss: 1.2410, Acc: 0.5231
Epoch 10, Loss: 1.2244, Acc: 0.5343
Epoch 11, Loss: 1.1895, Acc: 0.5460
Epoch 12, Loss: 1.1817, Acc: 0.5493
Epoch 13, Loss: 1.1691, Acc: 0.5537
Epoch 14, Loss: 1.1566, Acc: 0.5638
Epoch 15, Loss: 1.1472, Acc: 0.5659


In [6]:
model.eval()

val_loss, val_correct = 0.0, 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        val_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        val_correct += (preds == labels).sum().item()

val_loss /= len(val_loader)
val_acc = val_correct / len(val_loader.dataset)

print(f"\nFinal Validation Accuracy: {val_acc:.4f}, Validation Loss: {val_loss:.4f}")
wandb.log({"val_loss": val_loss, "val_acc": val_acc})

wandb.finish()


Final Validation Accuracy: 0.5775, Validation Loss: 1.1185


epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▃▅▅▆▆▇▇▇▇█████
train_loss,█▆▅▄▃▃▃▂▂▂▁▁▁▁▁
val_acc,▁
val_loss,▁
epoch,15
train_acc,0.56586
train_loss,1.14717
val_acc,0.5775
val_loss,1.11852
